In [ ]:
!mamba install -y -q pytorch=2.1.2 torchvision pytorch-cuda=12.1 scikit-learn h5py -c pytorch -c nvidia -c anaconda -c conda-forge
!pip install -q lightning torchmetrics pyts

In [ ]:
!nvidia-smi

In [ ]:
import h5py
import numpy as np
from pyts.image import RecurrencePlot
from pathlib import Path

import torch
from torch import nn, Tensor
from torch.nn import functional as F
from torch.utils.data import DataLoader, random_split, Dataset, SubsetRandomSampler
from torchvision import transforms as T
from torchvision.models import vgg11
import lightning as L
import torchmetrics
import pickle

from lightning.pytorch.callbacks.early_stopping import EarlyStopping

from sklearn import model_selection

# Optimization for GPU with Tensor cores
torch.set_float32_matmul_precision('medium')

SEED = 42
N_SPLITS = 20

class ActivityDataset(Dataset):
    def __init__(self, include_sparse_data=False, transform=None):
        self.fp = h5py.File('./ukdale-transformed-data.h5', swmr=True)
        self.activity = self.fp['activity'][:]
        
        self.transform = transform
        
        if not include_sparse_data:
            self.indices = np.argwhere(self.fp['activity'][:])
        else:
            self.indices = np.arange(len(self.activity), dtype=int)
            
        
    def __len__(self):
        return len(self.indices)
    
    def __getitem__(self, index):
        idx = self.indices[index]
        
        sample = self.fp['X'][idx]
        sample = np.squeeze(sample)
        sample = np.expand_dims(sample, axis=-1)
        
        label = self.fp['y'][idx]
        label = np.squeeze(label)
        
        if self.transform:
            sample = self.transform(sample)
        
        return sample, np.int64(label)
    
    def __del__(self):
        self.fp.close()

ActivityDataset(include_sparse_data=True)[0];
assert len(ActivityDataset(include_sparse_data=True)) > len(ActivityDataset(include_sparse_data=False))
print(len(ActivityDataset(include_sparse_data=True)), len(ActivityDataset(include_sparse_data=False)))

In [ ]:
# Labels in the dataset are numerical. Let's create mapper to meaningful names.
mapper = {
    value: idx
    for idx, value in
    enumerate(['HEKA', 'HTPC', 'boiler', 'computer monitor', 'desktop computer', 'fridge/freezer', 'laptop computer', 'light', 'microwave', 'server computer', 'television', 'washer dryer'])
}
reverse_mapper = { v:k for k, v in mapper.items() }

In [ ]:
num_classes = len(mapper); num_classes

In [ ]:
class Convert1to3channels:
    def __call__(self, x):
        return np.concatenate((x, x, x), axis=-1)
    
transform = T.Compose([
    #RecurrencePlotTransform(), # time-series to recurrence plot transformation
    Convert1to3channels(), # To fulfill CNN requirement of having 3 channels
    T.ToTensor(),
])


class Net(L.LightningModule):
    def __init__(self):
        super().__init__()
        self.net = vgg11()  # VGG11 (without BN) as a backbone
        
        # Loss metric
        self.criterion = nn.CrossEntropyLoss()
        
        # Evaluation metrics to see progress
        self.accuracy = torchmetrics.Accuracy(task='multiclass', num_classes=num_classes)
        
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        #x = torch.abs(x[:, :, :, None] - x[:, :, None, :]) # TODO: recurrencePlot on GPU?
        x = self.net(x)
        return x
    
    def predict_step(self, batch, batch_idx, dataloader_idx=0):
        inputs, _ = batch
        logits = self(inputs)
        return logits
        
    def training_step(self, batch, batch_idx):
        inputs, targets = batch
        logits = self(inputs)

        loss = self.criterion(logits, targets)
        self.log('train_loss', loss, on_step=True, on_epoch=True, prog_bar=True)
        return loss
    
    def validation_step(self, batch, batch_idx):
        inputs, targets = batch
        logits = self(inputs)
        loss = self.criterion(logits, targets)
        self.log('val_loss', loss, prog_bar=True)
        
        # validation metrics
        predictions = torch.argmax(logits, dim=1)
        acc = self.accuracy(predictions, targets)
        self.log('val_acc', acc, prog_bar=True)
        return loss
    
    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=1e-3)
        return optimizer


In [ ]:
import gc

dataloader_kwargs = dict(
    batch_size=128,
    num_workers=16,
    pin_memory=True,
    drop_last=False,
    #prefetch_factor=16,
    pin_memory_device='cuda',
)

kfold = model_selection.KFold(n_splits=N_SPLITS, shuffle=True, random_state=SEED)
#kfold = model_selection.StratifiedKFold(n_splits=N_SPLITS, shuffle=True, random_state=SEED)


CHECKPOINT_DIR = Path('./checkpoints/').resolve()
assert CHECKPOINT_DIR.exists()


results = {}
results["mapper"] = mapper
results["with-sparse"] = []
results["without-sparse"] = []


for USE_SPARSE in [True, False]: # switch between sparse and non-sparse dataset
    dataset = ActivityDataset(
        include_sparse_data=USE_SPARSE,
        transform=transform
    )
    
    
    for model_id, (train_ids, val_ids) in enumerate(kfold.split(dataset), start=1):
        checkpoint_path = CHECKPOINT_DIR / (f'model-{model_id+1:02}-sparse.ckpt' if USE_SPARSE else f'model-{model_id+1:02}.ckpt')
        
        if checkpoint_path.exists():
            continue
            
        gc.collect()
        torch.cuda.empty_cache()
        L.seed_everything(SEED + model_id)
        
        model = Net()
        
        train_loader = DataLoader(dataset, sampler=SubsetRandomSampler(train_ids), **dataloader_kwargs)
        val_loader = DataLoader(dataset, sampler=SubsetRandomSampler(val_ids), **dataloader_kwargs)
        
        trainer = L.Trainer(
            benchmark=True, # Enable GPU specific optimizations
            max_epochs=100,
            accelerator='gpu',
            devices=1,
            precision='bf16-mixed', # Mixed precision to speedup the process
            logger=False,
            enable_checkpointing=False,
            callbacks=[EarlyStopping(monitor='val_loss', min_delta=0.0, patience=3, mode='min', verbose=True)],
            #fast_dev_run=True
        )

        try:
            trainer.fit(model, train_loader, val_loader)
            trainer.save_checkpoint(checkpoint_path)
            print(f"{model_id+1} complete!")
        except RuntimeError: # in rare cases might fail
            pass

        outputs = trainer.predict(model, val_loader)
        #y_pred = torch.cat(y_pred).cpu().numpy()
        #break
        y_true = torch.cat([x[0] for x in outputs]).cpu().numpy()
        y_pred = torch.cat([x[1] for x in outputs]).cpu().numpy()
        y_proba = torch.cat([x[2] for x in outputs]).cpu().float().numpy()

        key = "with-sparse" if USE_SPARSE else "without-sparse"
        results[key].append(dict(y_true=y_true, y_pred=y_pred, y_proba=y_proba))
        
        del trainer, model


In [ ]:
joblib.dump(results, 'results.joblib', compress=0)